# Construcción de malla y otras utilidades de Gmsh

## Generación de la malla

Toda la malla está implementada en el archivo __*modelo_muestra.geo*__ que está en la carpeta *scripts*.

Para construir la geometría en Gmsh vamos a utilizar los dos kernels que contiene este software ("Built-in" y "OpenCascade") de manera combinada. Vamos a tratar de hacer todo en el script de tal forma que todas las instrucciones queden escritas (para no modificar, por ejemplo, la longitud característica máxima)
Los archivos de scripts para Gmsh tienen la extensión *.geo*, en el caso particular nuestro lo denominamos *modelo_muestra.geo*. Un script de Gmsh tiene la forma que se muestra en el siguiente [enlace](https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorial/t1.geo). En este tutorial nos saltearemos los conceptos básicos. 

Comenzamos directamente con el kernel OpenCascade construyendo un cilindro hueco (debe contener a toda la muestra). Este kernel permite crear todo tipo de volúmenes simples así como también intersecciones, uniones, difrencas, y otras operaciones entre ellos (no así el kernel "Built-in"). Entonces creamos un cilindro (para ver cómo crear cilindros y otras geometrías ver el siguiente [video](https://youtu.be/dywdlaaE1U8)). Las unidades las escribimos en mm porque la malla la generamos con esa unidad. Recordemos que la resolución del primer video fue de 0.03 mm.

In [ ]:
SetFactory("OpenCASCADE");
Cylinder(1) = {0, 0, -1.3, 0, 0, 2*1.3, 2.1, 2*Pi};
Delete { Volume{1}; } 

Notar que creamos un volumen y luego lo borramos, pero no borramos la superficie del mismo. Se puede ver a medida que voy escribiendo el script como va quedando en la interfaz gráfica que tiene Gmsh.

Luego le avisamos a Gmsh que utilice el otro kernel ("Built-in"). Lo necesitamos porque con este kernel se pueden combinar archivos de otras mallas y el cilindro que creamos antes.

In [ ]:
SetFactory("Built-in");
Merge "bone_meshlab.stl";

Con la función Merge levantamos la superficie teselada que ya hemos obtenido en el primer tutorial. Debe estar la misma carpeta que tenemos el script, le hemos puesto de nombre *bone_meshlab.stl*.

Tenemos hasta ahora varias superficies: las dos tapas y la cáscara circular del cilindro y otra que delimita la matriz ósea. Vamos a crear dos conjuntos de superficies.


In [ ]:
Surface Loop(100) = {1,2,3};//Cilindro
Surface Loop(200) = {4};//Hueso

Luego, creamos dos volúmenes: uno delimitado por la 200 (la matriz ósea) y el otro el resto menos la matriz ósea. Para escribirlo en Gmsh se hace así:

In [ ]:
marrow = newreg;
Volume(marrow) = {100,200}; // bone marrow
bone = newreg;
Volume(bone) = {200}; // bone

El comando *newreg* se utiliza para que Gmsh vaya enumerando solo las cosas y que luego pueda llamar esa región con el nombre y sea de más fácil lectura el código. Luego debemos definir y marcar que son cada cosa y cuáles son las superficies importantes (donde estará el voltaje positivo y la tierra). Esto lo hacemos mediante superficies y volúmenes físicos.


In [ ]:
Physical Surface(10) = {2};//electrodo activo
Physical Surface(20) = {3};//electrodo pasivo

Physical Volume(100) = {marrow};//marrow
Physical Volume(200) = {bone};//hueso

Finalmente, si bien la malla de la superficie del hueso ya está determinada por el teselado que importamos el resto no lo está, entonces es interesante fijar un tamaño máximo de elemento para que podamos informar esto en una publicación y para que tengamos idea de ese parámetro y comparar con otras simulaciones.

In [ ]:
Mesh.CharacteristicLengthMax = 0.1;

Luego para generar la malla podemos correr la instrucción:

In [ ]:
%%bash
gmsh -3 modelo_muestra.geo

O alternativamente podemos abrir la interfaz gráfica y mallarlo desde ahí. Es __importante__ que en el mallado no se genere ningún warning y obviamente ningún error, porque luego no se podrá importar para la resolución por MEF.

Una vez que la malla está generada podemos resolver por el método de elementos finitos. FEniCS puede leer mallas con extensión __*.xml*__ entonces el archivo __*.msh*__ generado antes debe ser convertido por el software *dolfin-convert* tool, puede obtenerse de [link](https://people.sc.fsu.edu/~jburkardt/py_src/dolfin-convert/dolfin-convert.html), aunque si la instalación de FEniCS funciona ya está instalado. Se convierte mediante la siguiente instrucción:

In [ ]:
%%bash
dolfin-convert modelo_muestra.msh modelo_muestra.xml

Esta conversión genera tres archivos: *modelo_muestra.xml*, *modelo_muestra_physical_region.xml*, y *modelo_muestra_facet_region.xml*, que contienen información de la malla, las marcas de los dominios, y las marcas de las fronteras o bordes, respectivamente.

En el siguiente [video](https://youtu.be/bfce-9XYB3k) les muestro algunas funcionalidades de la interfaz gráfica de Gmsh, que son útiles a la hora de ver si está haciendo lo que quiero en el script.

## Cálculo de volumen 

Con Gmsh también se puede computar el volumen de la muestra. Se utiliza un plugin que se llama *Integrate* y, en principio, puede integrar cualquier expresión, en este caso, como solo queremos el volumen integraremos solo 1. Previamente debemos crear un volumen que contenga solamente el hueso. Esto lo hacemos con las siguientes instrucciones (están en el archivo __*volumen_muestra.geo*__):

In [ ]:
SetFactory("Built-in");
 
Merge "bone_3dslicer.vtk";
Surface Loop(100) = {1};//Hueso
hueso = newreg;
Volume(hueso) = {100};
Mesh.CharacteristicLengthMax = 0.05;

Noten que sería interesante verificar la aproximación del cálculo de volumen según la cantidad de elementos que tenga el modelo. En este caso utilizamos el archivo "bone_3dslicer.vtk" que tiene una teselación con muchos más elementos. Luego ejecutamos el script:


In [ ]:
%%bash
gmsh -3 volumen_muestra.geo 

Y se generará el archivo __*volumen_muestra.msh*__, luego computamos el volumen con el script __*volumen.geo*__ que tiene las siguientes instrucciones.

In [ ]:
Merge "volumen_muestra.msh";
Plugin(NewView).Run;
Plugin(ModifyComponents).Expression0 = "1";
Plugin(ModifyComponents).Run;
Plugin(Integrate).Dimension = 3;
Plugin(Integrate).Run;

El resultado es:

Info    : Step 0: integral = 14.60829195337311

Luego, habrá que calcular el volumen del cilindro que lo contiene para obtener el parámetros de microestructura BV/TV.